In [1]:
import os
import numpy as np
import meshio
import subprocess
import pickle

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn import preprocessing


def create_gp_model(X,Y,Z,QoI,alpha):
    num_pts = X.shape[0]
    X_train_unscale = np.zeros((num_pts,3))
    X_train_unscale[:,0] = X
    X_train_unscale[:,1] = Y
    X_train_unscale[:,2] = Z
    scaler = preprocessing.StandardScaler().fit(X_train_unscale)
    X_train = scaler.transform(X_train_unscale)
    kernel = RationalQuadratic()
    gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha)
    gp.fit(X_train, QoI)
    return gp , scaler


def save_gp_model(gp_U,gp_V,gp_W,scaler,foldername):                            
    # saves the Gaussian process models to the specified folder                 
    pickle.dump(gp_U, open(os.path.join(foldername,'gp_U_cleaned.sav'),'wb'))   
    pickle.dump(gp_V, open(os.path.join(foldername,'gp_V_cleaned.sav'),'wb'))   
    pickle.dump(gp_W, open(os.path.join(foldername,'gp_W_cleaned.sav'),'wb'))   
    pickle.dump(scaler,open(os.path.join(foldername,'scaler_cleaned.sav'),'wb'))


def train_gpr(gp_dest_dir, mesh, nonspur_mask, alphas):
    X = mesh.points[nonspur_mask,0]
    Y = mesh.points[nonspur_mask,1]
    Z = mesh.points[nonspur_mask,2]
    U = mesh.point_data["u"][nonspur_mask,0]
    V = mesh.point_data["u"][nonspur_mask,1]
    W = mesh.point_data["u"][nonspur_mask,2]

    gp_U_cleaned, _ = create_gp_model(X,Y,Z,U,alphas[0])
    gp_V_cleaned, _ = create_gp_model(X,Y,Z,V,alphas[1])
    gp_W_cleaned,scaler_cleaned = create_gp_model(X,Y,Z,W,alphas[2])
    
    gp_stuff = (gp_U_cleaned,gp_V_cleaned,gp_W_cleaned,scaler_cleaned)

    try:
        if not os.path.exists(gp_dest_dir):
            os.makedirs(gp_dest_dir)
        save_gp_model(*gp_stuff, gp_dest_dir)
    except Exception:
        print("Encountered error attempting to save")
    
    return gp_stuff


_cache = dict()
def get_predicted_u_hmvic(gpr_path, points):
    # loads GPR models
    if gpr_path not in _cache:
        gp_U = pickle.load(open(os.path.join(gpr_path, 'gp_U_cleaned.sav'), 'rb'))
        gp_V = pickle.load(open(os.path.join(gpr_path, 'gp_V_cleaned.sav'), 'rb'))
        gp_W = pickle.load(open(os.path.join(gpr_path, 'gp_W_cleaned.sav'), 'rb'))
        scaler = pickle.load(open(os.path.join(gpr_path, 'scaler_cleaned.sav'),'rb'))
        _cache[gpr_path] = (gp_U, gp_V, gp_W, scaler)
    else:
        (gp_U, gp_V, gp_W, scaler) = _cache[gpr_path]

    input_arr = scaler.transform(points)

    N = 310000
    if len(input_arr) < N:
        u = gp_U.predict(input_arr)
        v = gp_V.predict(input_arr)
        w = gp_W.predict(input_arr)
        disp = np.column_stack((u,v,w))
    else:
        # Must stage it
        disp = np.zeros_like(input_arr)
        for batch_i in range(int(np.ceil(len(input_arr)/N))):
            u = gp_U.predict(input_arr[N*batch_i:N*(batch_i+1)])
            v = gp_V.predict(input_arr[N*batch_i:N*(batch_i+1)])
            w = gp_W.predict(input_arr[N*batch_i:N*(batch_i+1)])
            disp[N*batch_i:N*(batch_i+1)] = np.column_stack((u,v,w))

    return disp

This notebook is not meant to be run in the same Python environment as the inverse model. Please use an environment suitable for the object-oriented version of [FM-Track](https://github.com/elejeune11/FM-Track/tree/objectoriented).

See the fenics_environment_analysis.ipynb notebook for guidance on obtaining data, and change directory to it below:

In [ ]:
%cd YOUR_DATA_DIR_HERE

# GPR Interpolation of Noisy Micro-Sphere Displacements

Here we use Gaussian Process Regression to interplate ground-truth displacements generated with a forward simulation with a finer mesh, and then synthetic noise was added, onto a coarser mesh. This emulates the case of real noisy micro-sphere data.

In [3]:
noise_levels = "noisy_target_test/noise_lvls.txt"
alphas = np.loadtxt(noise_levels)**2

In [4]:
dest_dir = "noisy_target_test"
cell_data_dir = "cell_data_A"

In [5]:
def prepare_test_case(alpha_stiff_amp):
    bead_init = np.loadtxt(os.path.join(dest_dir, f"bead_init_{alpha_stiff_amp}_nic.txt"))
    bead_u = np.loadtxt(os.path.join(dest_dir, f"bead_u_{alpha_stiff_amp}_nic.txt"))

    pm_bead_mesh = meshio.Mesh(bead_init, [], point_data = {"u" : bead_u})

    gp_dest_dir = os.path.join(dest_dir, f"gp_denoise_{alpha_stiff_amp}_nic")

    gp_stuff = train_gpr(gp_dest_dir, pm_bead_mesh, np.ones(len(pm_bead_mesh.points)).astype(bool), alphas)
    print("Finished GPR")
    
    return gp_dest_dir

In [6]:
alpha_stiff_amp = 1000

gp_dest_dir = prepare_test_case(alpha_stiff_amp)

Finished GPR


In [8]:
init_points = np.loadtxt(os.path.join(cell_data_dir, "cell_vertices_initial.txt"))
u = get_predicted_u_hmvic(gp_dest_dir, init_points)
cell_mesh = meshio.Mesh(init_points, {}, point_data={"u":u})
cell_mesh.write(os.path.join(dest_dir, "bci.vtk"))

gel_mesh_boundaries = meshio.read(os.path.join(cell_data_dir, "reference_domain_boundaries.xdmf"))
on_outer_boundary = np.zeros(len(gel_mesh_boundaries.points)).astype(bool)
on_outer_boundary[gel_mesh_boundaries.cells[0].data[gel_mesh_boundaries.cell_data["boundaries"][0] == 201]] = True
bndry_points = gel_mesh_boundaries.points[on_outer_boundary]
u = get_predicted_u_hmvic(gp_dest_dir, bndry_points)
cube_mesh = meshio.Mesh(bndry_points, {}, point_data={"u":u})
cube_mesh.write(os.path.join(dest_dir, "bco.vtk"))

# get full shape u field
conda_env = os.path.join(
    os.path.dirname(os.environ["CONDA_PREFIX"]),
    "newestfenics"
)
output_file = os.path.join(dest_dir, f'u_exp_{alpha_stiff_amp}.xdmf')
script_cmdline = f"conda run -p {conda_env} get_u -c {cell_data_dir} -g {gp_dest_dir} -o {output_file}"
subprocess.run(script_cmdline, shell=True)

CompletedProcess(args='conda run -p /home/gpeery/miniconda3/envs/newestfenics get_u -c cell_data_A -g noisy_target_test/gp_denoise_1000_nic -o noisy_target_test/check/u_exp_1000.xdmf', returncode=0)